# Power Packs



In [34]:
import math

def solve_relationship(relation, values):
    """Solve a=b*c, a=b²*c, or a=b²/c relationship given some values"""
    a_key, b_key, c_key, rel_type = relation
    a, b, c = values.get(a_key), values.get(b_key), values.get(c_key)
    
    provided = sum(x is not None for x in [a, b, c])
    if provided < 2:
        return {}
    
    # Calculate missing values based on relationship type
    if rel_type == "multiply":  # a = b * c
        if a is None: a = b * c
        elif b is None: b = a / c
        elif c is None: c = a / b
    
    elif rel_type == "square_multiply":  # a = b² * c  
        if a is None: a = b**2 * c
        elif b is None: b = math.sqrt(a / c)
        elif c is None: c = a / b**2
    
    elif rel_type == "square_divide":  # a = b² / c
        if a is None: a = b**2 / c
        elif b is None: b = math.sqrt(a * c)  
        elif c is None: c = b**2 / a
    
    return {a_key: a, b_key: b, c_key: c}

def ohm(voltage=None, resistance=None, current=None, power=None):
    """Ohm's Law Calculator using structural relationships"""
    
    # Define all Ohm's law relationships structurally
    relationships = [
        ('voltage', 'current', 'resistance', 'multiply'),      # V = I * R
        ('power', 'current', 'resistance', 'square_multiply'), # P = I² * R  
        ('power', 'voltage', 'resistance', 'square_divide'),   # P = V² / R
        ('power', 'voltage', 'current', 'multiply'),           # P = V * I
    ]
    
    values = {'voltage': voltage, 'resistance': resistance, 'current': current, 'power': power}
    provided_count = sum(v is not None for v in values.values())
    
    if provided_count < 2:
        return {**values, 'valid': True, 'sufficient': False}
    
    # Try each relationship to fill in missing values
    try:
        for relation in relationships:
            result = solve_relationship(relation, values)
            if result:  # If this relationship could contribute new values
                values.update(result)
        
        # Validate: check if all relationships hold
        v, r, i, p = values['voltage'], values['resistance'], values['current'], values['power']
        if all(x is not None for x in [v, r, i, p]):
            tolerance = 1e-10
            valid = (abs(v - i * r) < tolerance and 
                    abs(p - i**2 * r) < tolerance and
                    abs(p - v**2 / r) < tolerance and 
                    abs(p - v * i) < tolerance)
        else:
            valid = True  # Can't validate incomplete results
            
        return {**values, 'valid': valid}
        
    except (ZeroDivisionError, ValueError):
        return {**values, 'valid': False}


# Test the structural approach
def run_tests():
    print("STRUCTURAL OHMS LAW CALCULATOR")
    print("=" * 40)
    
    test_cases = [
        ("V=I*R: solve for P", {'voltage': 6, 'current': 2, 'resistance': 3}),
        ("P=I²*R: solve for V", {'power': 8, 'current': 2, 'resistance': 2}),
        ("P=V²/R: solve for I", {'power': 25, 'voltage': 10, 'resistance': 4}),
        ("P=V*I: solve for R", {'power': 12, 'voltage': 4, 'current': 3}),
        ("Two values: P,I", {'power': 18, 'current': 3}),
        ("Validation: all correct", {'voltage': 2, 'resistance': 1, 'current': 2, 'power': 4}),
        ("Validation: inconsistent", {'voltage': 5, 'resistance': 2, 'current': 1, 'power': 10}),
    ]
    
    for name, params in test_cases:
        result = ohm(**params)
        print(f"\n{name}: {params}")
        print(f"→ {result}")

run_tests()

STRUCTURAL OHMS LAW CALCULATOR

V=I*R: solve for P: {'voltage': 6, 'current': 2, 'resistance': 3}
→ {'voltage': 6, 'resistance': 3, 'current': 2, 'power': 12, 'valid': True}

P=I²*R: solve for V: {'power': 8, 'current': 2, 'resistance': 2}
→ {'voltage': 4, 'resistance': 2, 'current': 2, 'power': 8, 'valid': True}

P=V²/R: solve for I: {'power': 25, 'voltage': 10, 'resistance': 4}
→ {'voltage': 10, 'resistance': 4, 'current': 2.5, 'power': 25, 'valid': True}

P=V*I: solve for R: {'power': 12, 'voltage': 4, 'current': 3}
→ {'voltage': 4, 'resistance': 1.3333333333333333, 'current': 3, 'power': 12, 'valid': True}

Two values: P,I: {'power': 18, 'current': 3}
→ {'voltage': 6.0, 'resistance': 2.0, 'current': 3, 'power': 18, 'valid': True}

Validation: all correct: {'voltage': 2, 'resistance': 1, 'current': 2, 'power': 4}
→ {'voltage': 2, 'resistance': 1, 'current': 2, 'power': 4, 'valid': True}

Validation: inconsistent: {'voltage': 5, 'resistance': 2, 'current': 1, 'power': 10}
→ {'voltage